In [95]:
import simpy
import random


The first step would be to implement the ressources along with its attributes and methods
### 1. Ressources
* Each resource belongs to an environment
* Each resource has a capacity, so a simpy.Container()
  * A Method to check the capacity
  * a trigger that book/pick and 
  * releases/restock the capacity

In [96]:
#Recssources are:
#   - Paper 
#   - robot
#   - Operator
#   - Jig1 -> like payment in the example
#   - Jig2 -> like payment in the example

#1 Define a paper class
class paper(object):
    # Further container attributes required?
    def __init__(self, env, station3):
        self.env = env
        self.station3 = station3

        # Pickup Station 3 is a container with a capacity of 1 and initial value of 1
        self.pickup_station2 = simpy.Container(env, init=1, capacity=1)
        # at the beginning the pickup area is filled
        self.pickup_station2.put(1)
   

    # Check if paper is available in pickup area
    def check_availability_pickup_station2(self, robot):
        # to start the process the pickup area has to be filled
        # check if the process can be started or if the pickup area is empty

        #this takes between 1 seconds and 5 seconds
        yield self.env.timeout(random.randint(1, 5))

        # if the pickup area is filled the process can start
        if self.pickup_station2.level == 1:
            # if the paper is available, return True
            # print("Paper is available")
            print("Paper is available")
            return True
        else:
            print("There is no paper available, please refill the pickup area")
            # TODO: trigger a loop to check if the pickup area is refilled -> done see [def pick_paper]
            return False


    # Trigger the pickup process and reduce the pickup area by 1 
    def pick_paper(self, robot, station3):
        # to start the process the paper of station 2 has to be filled
        # check if the process can be started or if the pickup area is empty
        while not self.env.process(self.check_availability_pickup_station2(robot)):
            # if the pickup area is empty, wait until it is refilled
            yield self.env.timeout(1)  # wait for 1 second before checking again
        # pick up the paper
        yield self.pickup_station2.get(1) & self.env.process(self.restock()) 
        
    
    def restock_station3_by_station2(self, station3):
        #restock the pickup area, this takes between 60 second and (cycle time of station 1&2?) selected e.g. 100 seconds
        yield self.env.timeout(random.randint(60,100))
        # restock the pickup area
        yield self.pickup_station2.put(1)

# ---------------------------------------------
    

#2 Define a robot class
class robot(object):
    def __init__(self, env, station3):
        self.env = env
        # the robot belongs to station 3
        self.station3 = station3
        # code the robot as a container/resource with a capacity of 1
        self.robot = simpy.Resource(env, capacity=1)
        self.quality_check_jig1 = False #Quality check of jig1 process is not done TODO: is this required?
        self.quality_check_jig2 = False #Quality check of jig2 process is not done TODO: is this required?
        # at the beginning the robot is available
        self.available = True
        # at the beginning the robot is not at the start position
        self.at_start_position = False
        # at the begonning is no paper loaded
        self.paper_loaded = False
    

# Actuate the robot, check if the robot is available and start position of the robot is reached. If yes, start the process of the robot, else wait until the robot is available and the start position is reached.
    def actuate(self, station3):
        while True:
            try:
                # Check if the robot is available and the start position is reached
                if self.available and self.at_start_position:
                    # start the process of the robot
                    self.env.process(self.start_process())
                    # the actuation and check takes between 1 second and 6 seconds
                    yield self.env.timeout(random.randint(1, 6))
                    # the robot is not available anymore
                    self.available = False
                else:
                    # wait until the robot is available and the start position is reached
                    yield self.env.timeout(1)  # Warte 1 Sekunde, bevor erneut geprüft wird
            except Exception as e: # Error message if the robot is not available or the start position is not reached
                print(f"Error: {e}")   
                yield self.env.process(self.eliminate_actuation_error(self)) # Link to class operator def eliminate_actuation_error if there is an Error

    def start_process(self):
        # Pocesslogic of the robot
        pass

# load the paper from station 2 if this is available in pick up area of station 2
    def load_paper(self, paper, station2, station3):
        # to load the paper takes between 2.5 seconds and 5 seconds
        yield self.env.timeout(random.randint(2.5, 5))
        # check if the paper is available in the pickup area of station 2 
        yield self.env.process(paper.pick_paper(self, self.station3)) #Link to class paper def pick_paper
        # if the paper is available, load the paper
        print("Paper is loaded")
        self.paper_loaded = True
 
#check if the paper is loaded by sensor or camera 
    def check_paper_loaded(self):
        # this takes between 2.5 seconds and 5 seconds
        yield self.env.timeout(random.randint(2.5, 5))
        if not self.paper_loaded:
            raise Exception("Error: Paper not correctly loaded")
            yield self.env.process(self.eliminate_paper_loading_error_robot(self)) # Link to class operator def eliminate_paper_loading_error_robot if there is an Error
        else:
            pass

# push the paper to the Jig1 by robot
    def push_paper_to_jig1(self, jig1):
        # to push the paper to jig1 takes between 5 seconds and 10 seconds
        yield self.env.timeout(random.randint(5, 10))
        yield self.env.process(jig1.folding_process_1(self))    # Link to class jig1 def folding_process_1

# insert the paper to Jig2 by robot 
    def insert_paper_to_jig2(self, jig2):
        # to insert the paper to jig2 takes between 5 seconds and 10 seconds
        yield self.env.timeout(random.randint(5, 10))
        yield self.env.process(jig2.check_paper_availability_jig2(self)) # link to class jig2 def check_paper_availability_jig2
        # yield self.env.process(jig2.folding_process_2(self))    # TODO: sequence of process is done later in class station 3 def process_station3 -> done

# extract the paper from Jig2 by robot
    def extract_paper_from_jig2(self, jig2):
        # to extract the paper from jig2 takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))

# hand over the paper to station 4 by robot
    def hand_over_paper(self, station4):
        # to hand over the paper takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))
        # after the hand over the robot is unloaded 
        self.paper_loaded = False
        # check if the paper is unloaded by robot
        yield self.env.process(self.check_paper_unloaded(self))

# check if the paper is unloaded by sensor or camera
    def check_paper_unloaded(self):
        if self.paper_loaded == True:
            yield self.env.process(self.eliminate_hand_over_error(self)) # link to class operator def eliminate_hand_over_error if there is an unloading error
            raise Exception("Error: Paper not correctly unloaded")
        else:
            pass

# Robot has to return to start position after the hand over of the paper
    def return_to_start(self):
        # Überprüfen Sie, ob das Papier entladen wurde, bevor Sie zur Startposition zurückkehren
        self.check_paper_unloaded()
        # to return to start position takes 10 seconds
        yield self.env.timeout(10)
        # Zurück zur Startposition
        self.at_start_position = True
        print("Roboter ist zurück zur Startposition gefahren")
        # Der Roboter ist wieder verfügbar
        self.available = True
    
# ---------------------------------------------
    

#3 Define a operator class

class operator(object):
    def __init__(self, env, station3, quality_check_jig1, quality_check_jig2):
        self.env = env
        self.quality_check_jig1 = quality_check_jig1 
        self.quality_check_jig2 = quality_check_jig2 
        # the operator belongs to station 3
        self.station3 = station3
        # code the operator as a container/resource with a capacity of 1
        self.operator = simpy.Resource(env, capacity=1)
        # at the beginning the operator is available
        self.available = True
    
    
# Elimate error of actuation of the robot
    def eliminate_actuation_error(self, robot):
        # to report the error takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))
        # to eliminate the error takes between 15 seconds and 30 seconds
        yield self.env.timeout(random.randint(15, 30))
        # to confirm the elimination of the error takes between 5 seconds and 10 seconds
        yield self.env.timeout(random.randint(5, 10))
        # after the error is eliminated the robot is available again # TODO: required?
        robot.available = True
        # after the error is eliminated the robot is at start position again
        robot.at_start_position = True
        print("Error is eliminated and robot is at start position again")
        yield self.env.process(robot.actuate(self)) # Link to class robot def actuate (Loop)

# Elimate error of robot loading the paper
    def eliminate_paper_loading_error_robot(self, robot):
        # to report the error takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))
        # to eliminate the error takes between 15 seconds and 30 seconds
        yield self.env.timeout(random.randint(15, 30))
        # to confirm the elimination of the error takes between 5 seconds and 10 seconds
        yield self.env.timeout(random.randint(5, 10))
        # after the error is eliminated the robot is available again
        robot.available = True 
        print("Error is eliminated and robot is ready to push the paper to jig1")
        yield self.env.process(robot.load_paper(self)) # Link to class robot to load paper (Loop); 

# qulity check of jig1 process
    def quality_check_jig1(self, jig1, quality1):
       # the quality check takes between 2.5 second and 5 seconds
        yield self.env.timeout(random.randint(2.5, 5))
        # redomly decide if the quality check is ok or not
        quality1 = random.choice([True, False])
        if quality1 == False:
            # operator has to adjust the paper, this takes between 10 seconds and 15 seconds
            yield self.env.timeout(random.randint(10, 15))
            yield self.env.process(robot.push_paper_to_jig1(self)) # link to class robot to push the paper to jig1 (Loop)
        else:
            # if the quality check is ok, continue with the next process
            pass
        self.quality_check_jig1 = True #Quality check of jig1 process is done

# Elimate error of loading the paper to jig2
    def eliminate_paper_loading_error_jig2(self, robot):
        # to report the error takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))
        # to eliminate the error takes between 15 seconds and 30 seconds
        yield self.env.timeout(random.randint(15, 30))
        # to confirm the elimination of the error takes between 2.5 seconds and 5 seconds
        yield self.env.timeout(random.randint(2.5, 5))
        # after the error is eliminated the jig2 is ready to double check if the paper is available
        yield self.env.process(jig2.check_paper_availability_jig2(self)) # Link to class jig2  def  check_paper_availability_jig2 (Loop)

# qulity check of jig2 process
    def quality_check_jig2(self, jig2, quality2): # TODO: oder verweisen auf quality_check_jig2 in class robot?
       # the quality check takes between 2.5 second and 5 seconds
        yield self.env.timeout(random.randint(2.5, 5))
        # redomly decide if the quality check is ok or not
        self.quality2 = random.choice([True, False])
        if quality2 == False:
            # operator has to adjust the paper, this takes between 10 seconds and 15 seconds
            yield self.env.timeout(random.randint(10, 15))
            yield self.env.process(self.quality_check_jig2(self)) # link to class operator def quality_check_jig2 (Loop)
        
        else:
            # if the quality check is ok, continue with the next process
            pass
        self.quality_check_jig2 = True #Quality check of jig2 process is done

# Elimate error of hand over the paper
    def eliminate_hand_over_error(self, robot):
        # to report the error takes between 1 second and 5 seconds
        yield self.env.timeout(random.randint(1, 5))
        # to eliminate the error takes between 5 seconds and 10 seconds
        yield self.env.timeout(random.randint(5, 10))
        # to confirm the elimination of the error takes between 1 second and 15 seconds
        yield self.env.timeout(random.randint(1, 5))
        # after the error is eliminated the robot is available again 
        robot.available = True
        # after the error is eliminated the robot is at start position again
        robot.at_start_position = True 

# ---------------------------------------------


#4 Define a jig1 class
class jig1(object):
    def __init__(self, env, station3):
        self.env = env
        # the jig1 belongs to station 3
        self.station3 = station3
        # code the jig1 as a container/resource with a capacity of 1 and initial value of 0
        self.jig1 = simpy.Resource(env, 1)
        # at the beginning the jig1 is available
        self.available = True
    
# folding process 1 of jig1
    def folding_process_1(self, robot):
        # to fold the paper takes 10 seconds
        yield self.env.timeout(10)
        # while the folding process 1 is running, the jig1 is not available
        self.available = False
    
# ---------------------------------------------


#5 Define a jig2 class

class jig2(object):
    def __init__(self, env, station3):
        self.env = env
        # the jig2 belongs to station 3
        self.station3 = station3
        # code the jig2 as a container/resource with a capacity of 1 and initial value of 0
        self.jig2 = simpy.Resource(env, 1)
        # at the beginning the jig2 is available
        self.available = True
        # at the beginning the jig2 is not filled
        self.filled = False

# check if the paper is available in jig2
    def check_paper_availability_jig2(self, robot):
        # to start the process the jig2 has to be filled
        # check if the process can be started or if the jig2 is empty
        yield self.env.timeout(10)
        if not robot.insert_paper_to_jig2: 
            yield self.env.process(self.eliminate_paper_loading_error_jig2(self)) # link to class operator def eliminate_paper_loading_error_jig2 if there is an Error
            raise Exception("Error: Paper not correctly inserted to Jig2")
        else:
            yield self.env.process(self.folding_process_2(self)) # link to class jig2 def folding_process_2 (Loop)

# folding process 2 of jig2
    def folding_process_2(self, robot):
        # to fold the paper takes 10 seconds
        yield self.env.timeout(10)
        # while the folding process 2 is running, the jig2 is not available
        self.available = False

# ---------------------------------------------
        
        

In the 2nd step we will implement the environment in which the simulation takes place.
Here we will set up the ressources and the genereal process flow.
### 2.Environment

TODO: Our first investigation will be on different customer demands. So we create a little method to generate a customer queue - required?

In [97]:
#Environment is:
#   - Station 3

#6 Define a station3 class and define the process parameters and the process logic
class Station3(object):
    def __init__(self, env, paper_list):
        self.env = env
        self.paper_list = paper_list
        # There is one robot available:
        self.robot = robot(env, self)
        # There are operators available. Start with 1 operator:
        self.operator = operator(env, self, self.robot.quality_check_jig1, self.robot.quality_check_jig2)
        # There is one jig1 available:
        self.jig1 = jig1(env, self)
        # There is one jig2 available:
        self.jig2 = jig2(env, self)
        # There is one paper available: 
        self.paper = paper(env, self)
        # count the number of produced papers
        self.produced_papers = 0
        # count the total production time
        self.total_production_time = 0
        # TODO: Change here if you want to create a fixed number of customers
        # at the beginning quality 1 and quality 2 are not done
        self.quality1 = False
        self.quality2 = False
        
# ---------------------------------------------

# Now define the process logic of station 3 to produce one paper
    def station3_process(self, paper): # TODO: is paper correct?
        # track the time of the process
        start_time = self.env.now
        # First the robot has to actuate, the check of the starting position is included in this step # TODO: or start with the availability of the paper?
        yield self.env.process(self.robot.actuate(self))
        print ("Paper "+str(self.produced_papers)+" started the process at "+str(self.env.now)+".")
        # Second the robot has to load the paper from station 2 and check if the paper is loaded
        yield self.env.process(self.robot.load_paper(self.paper, self, self))
        print ("Robot "+str(self.produced_papers)+" loaded the paper at "+str(self.env.now)+".")
        yield self.env.process(self.robot.check_paper_loaded(self))
        print ("Robot "+str(self.produced_papers)+" checked the paper loaded at "+str(self.env.now)+".")
        # Third the robot has to push the paper to jig1
        yield self.env.process(self.robot.push_paper_to_jig1(self.jig1))
        print ("Robot pushed the paper"+str(self.produced_papers)+" to jig1 at "+str(self.env.now)+".")
        # Fourth the operator has to check the quality of jig1
        yield self.env.process(self.operator.quality_check_jig1(self.jig1, self.robot.quality1))
        print ("Operator checked the quality of fold 1 of the paper "+str(self.produced_papers)+" at "+str(self.env.now)+".")
        # Fifth the robot has to insert the paper to jig2
        yield self.env.process(self.robot.insert_paper_to_jig2(self.jig2))
        print ("Robot inserted the paper "+str(self.produced_papers)+" to jig2 at "+str(self.env.now)+".")
        # sixth the robot has to extract the paper from jig2
        yield self.env.process(self.robot.extract_paper_from_jig2(self.jig2))
        print ("Robot extracted the paper "+str(self.produced_papers)+" from jig2 at "+str(self.env.now)+".")
        # seventh the operator has to check the quality of jig2
        yield self.env.process(self.operator.quality_check_jig2(self.jig2, self.robot.quality2))
        print ("Operator checked the quality of fold 2 of the paper "+str(self.produced_papers)+" at "+str(self.env.now)+".")
        # eighth the robot has to hand over the paper to station 4
        yield self.env.process(self.robot.hand_over_paper(self))
        print ("Robot handed over the paper "+str(self.produced_papers)+" to station 4 at "+str(self.env.now)+".")
        # ninth the robot has to return to start position
        yield self.env.process(self.robot.return_to_start(self))
        print ("Robot returned to start position after paper "+str(self.produced_papers)+" at "+str(self.env.now)+".")
        # tenth the process is finished and the robot is ready for the next paper
        yield self.env.process(self.robot.actuate(self))
        # track the time of the process
        process_time = self.env.now - start_time
        paper.process_time = process_time
        # print ("Process time of "+str(self.produced_papers)+" is "+str(self.env.now)+" seconds.")
        print ("Paper "+str(self.produced_papers)+" left station 3 after "+str(self.env.now)+" seconds.") 
        # add the process time to the total production time
        total_production_time += process_time
        # count the produced papers
        self.produced_papers += 1  
        
        
        

In the third step we will run the simulation and therefor define an initial set of parameter to simulate.
### 3. Simulation

Now we need to run the simulation

In [98]:
# The sequence of the availablity of the paper depends on the station 1&2 process.
# For this simulation we have to code a generator to generate the paper in different time intervals
def paper_generator(env, station3, paper_list):
    # generate more papers while the simulation is running
    while True:
        # interarrival time of the papers is between 60 seconds and 100 seconds
        interarrival_time = random.randint(60, 100)
        # create a new paper
        paper_generator1 = paper(env, station3)
        paper_list.append(paper_generator1)
        # CHAT increment the count of produced papers (following line)
        station3.produced_papers += 1
        yield env.timeout(interarrival_time)
        env.process(station3.station3_process(paper))
        # TEST print the finishing time of the process
        # print("Paper "+str(station3.produced_papers)+" finished the process at "+str(env.now)+" seconds.")
        # wait for the next paper !: is already included in the paper generator because it's running parallel
        # yield env.timeout(interarrival_time)
        print("Next Paper "+str(station3.produced_papers)+" arrived at station 3 pickup area at "+str(env.now)+" seconds.")

# ---------------------------------------------
        # TODO: create seconde generator with fixed number of papers


In [99]:
# First create the environment
env = simpy.Environment()
# Then create the station 
Station_instance = Station3(env, [])
# Create a list for the papers
paper_list = []
# select the generator for the paper arrival
env.process(paper_generator(env, Station_instance, paper_list))
# Run the simulation for 8 hours (28800 seconds) - 1 shift 
env.run(until=28800)

# ---------------------------------------------
# The siemulation is finished, now we can analyse the results
print ("The simulation is finished")
# Print the time the simulation runs
print ("The simulation runs for "+str(env.now)+" seconds.")
# Print the number of produced papers
# Keep in mind, that there is a paper in the process at the beginning of the simulation
print ("The number of produced papers is "+str(Station_instance.produced_papers-1)+".")
# Average process time of the produced papers TEST: Station 3_PROCESS
# print ("The average process time of the produced papers is "+str(Station_instance.total_production_time/(Station_instance.produced_papers-1))+" seconds.")



Next Paper 1 arrived at station 3 pickup area at 80 seconds.
Next Paper 2 arrived at station 3 pickup area at 158 seconds.
Next Paper 3 arrived at station 3 pickup area at 227 seconds.
Next Paper 4 arrived at station 3 pickup area at 306 seconds.
Next Paper 5 arrived at station 3 pickup area at 402 seconds.
Next Paper 6 arrived at station 3 pickup area at 485 seconds.
Next Paper 7 arrived at station 3 pickup area at 549 seconds.
Next Paper 8 arrived at station 3 pickup area at 629 seconds.
Next Paper 9 arrived at station 3 pickup area at 703 seconds.
Next Paper 10 arrived at station 3 pickup area at 789 seconds.
Next Paper 11 arrived at station 3 pickup area at 868 seconds.
Next Paper 12 arrived at station 3 pickup area at 952 seconds.
Next Paper 13 arrived at station 3 pickup area at 1048 seconds.
Next Paper 14 arrived at station 3 pickup area at 1126 seconds.
Next Paper 15 arrived at station 3 pickup area at 1190 seconds.
Next Paper 16 arrived at station 3 pickup area at 1270 seconds

In [100]:
# calculate the average process time of the produced papers
number_of_produces_papers = len(paper_list)
simulation_term = 28800
average_process_time = simulation_term/number_of_produces_papers
print ("The average process time of the produced papers is {:.2f} seconds.".format(average_process_time))

The average process time of the produced papers is 80.67 seconds.
